In [1]:
# If not installed, run this cell. It is needed to read DBF files
#!pip install dbfread
#!pip show dbfread

In [2]:
import arcpy, pandas as pd
from arcpy import env
from arcpy.sa import *
from dbfread import DBF

In [3]:
# Paths to input datasets
root_folder = r"Z:\PhD_Datasets&Analysis\Info_Inputs"
tc_ds = root_folder + "\\TerraClimate"
str_flows_folder = root_folder + "\\Streamflow_Stations\\_DataFrames"
drain_areas_folder = root_folder + "\\Streamflow_Sts_Drainage_Areas"
bands_gee = ["pr", "pet", "ro"] # band names in gee
tc_vars = ["ppt", "pet", "q"] # variable names according to TerraClimate


# Set arcpy environment variables
env.overwriteOutput = True
arcpy.CheckOutExtension("spatial")
env.cellSize = "MINOF"

In [4]:
# Limit the spatil processing extent
buffer_zoi = drain_areas_folder + "\\Calibration_Basins_Final_Generalized_Buffer.shp"
env.mask = buffer_zoi
env.extent = buffer_zoi

In [5]:
# Dataframe with streamflow datasets
str_flows = pd.read_csv(str_flows_folder + "\\Joined_Sts_DFs.csv")
str_flows

,yr-mth,MISS_07374525,ULUA_23,AMZN_17050001,NILE_1662100,YNGZ_2181900,MUDA_A4261110,DABE_6742900,DABE_42055
0,1958-1,NaN,NaN,NaN,866.0,8770.0,NaN,5113.0,NaN
1,1958-2,NaN,NaN,NaN,777.0,9400.0,NaN,4995.0,NaN
2,1958-3,NaN,NaN,NaN,810.0,12000.0,NaN,8735.0,NaN
3,1958-4,NaN,NaN,NaN,925.0,22900.0,NaN,10573.0,NaN
4,1958-5,NaN,NaN,NaN,660.0,45600.0,NaN,11352.0,NaN
...,...,...,...,...,...,...,...,...,...
787,2023-8,7495.469295,NaN,NaN,NaN,NaN,449.813,NaN,NaN
788,2023-9,4442.913232,NaN,NaN,NaN,NaN,335.637,NaN,NaN
789,2023-10,NaN,NaN,NaN,NaN,NaN,199.097,NaN,NaN
790,2023-11,4935.626362,NaN,NaN,NaN,NaN,212.866,NaN,NaN


In [6]:
# Read the DBF file
drain_areas_tb = DBF(drain_areas_folder + "\\Calibration_Basins_Final_Generalized.dbf")

# Convert to DataFrame
drain_areas_df = pd.DataFrame(iter(drain_areas_tb)).sort_values(by='gridcode')
drain_areas_df

,gridcode,Area_km2,Length_km,ID,STATION,SUFIX,BASIN,LAT,LONG,CONTINENT
2,0,3243820,14815,07374525,USGS 07374525 Mississippi River at Belle Chass...,MISS,Mississippi,29.857145,-89.977850,na
3,1,20234,841,23,Santiago,ULUA,Ulua,15.297001,-87.928217,na
4,2,4671618,15275,17050001,Obidos,AMZN,Amazon,-1.947222,-55.511108,sa
5,3,2606299,12473,1662100,Dongola,NILE,Nile,19.186000,30.489500,af
6,4,1693268,11832,2181900,Datong,YNGZ,Yangtze,30.770000,117.620000,as
7,5,975286,6523,A4261110,River Murray downstream Morgan (AMTD 316.5km),MUDA,Murray-Darling,-34.055059,139.685774,au
0,6,788358,6820,6742900,Ceatal Izmail,DABE,Danube,45.216667,28.716657,eu
1,7,520313,5911,42055,Smederevo,DABE,Danube,44.666780,20.920600,eu


In [7]:
drain_areas_df["SUFIX_ID"] = drain_areas_df["SUFIX"] + "_" + drain_areas_df["ID"]
drain_areas_df

,gridcode,Area_km2,Length_km,ID,STATION,SUFIX,BASIN,LAT,LONG,CONTINENT,SUFIX_ID
2,0,3243820,14815,07374525,USGS 07374525 Mississippi River at Belle Chass...,MISS,Mississippi,29.857145,-89.977850,na,MISS_07374525
3,1,20234,841,23,Santiago,ULUA,Ulua,15.297001,-87.928217,na,ULUA_23
4,2,4671618,15275,17050001,Obidos,AMZN,Amazon,-1.947222,-55.511108,sa,AMZN_17050001
5,3,2606299,12473,1662100,Dongola,NILE,Nile,19.186000,30.489500,af,NILE_1662100
6,4,1693268,11832,2181900,Datong,YNGZ,Yangtze,30.770000,117.620000,as,YNGZ_2181900
7,5,975286,6523,A4261110,River Murray downstream Morgan (AMTD 316.5km),MUDA,Murray-Darling,-34.055059,139.685774,au,MUDA_A4261110
0,6,788358,6820,6742900,Ceatal Izmail,DABE,Danube,45.216667,28.716657,eu,DABE_6742900
1,7,520313,5911,42055,Smederevo,DABE,Danube,44.666780,20.920600,eu,DABE_42055


In [8]:
# Creating dictionary with ids of the basins
id_dict = dict(zip(drain_areas_df['gridcode'], drain_areas_df['SUFIX_ID']))
id_dict

{0: 'MISS_07374525',
 1: 'ULUA_23',
 2: 'AMZN_17050001',
 3: 'NILE_1662100',
 4: 'YNGZ_2181900',
 5: 'MUDA_A4261110',
 6: 'DABE_6742900',
 7: 'DABE_42055'}

In [9]:
whc = Float(Raster("Z:\PhD_Datasets&Analysis\Inputs\WHC_FC33_Final.tif")) / 1000 # As the raster was originally multiplied by 1000
drain_areas_lyr = arcpy.MakeFeatureLayer_management(drain_areas_folder + "\\Calibration_Basins_Final_Generalized.shp")
drain_areas_ids = Raster(drain_areas_folder + "\\calib_ba_fin_gen.tif")

In [10]:
######################################################
### Starting values for the water balance model - T&M
######################################################

# Initial variables
warmup_yrs = 5
years = range(1958, 2023 + 1) # Years with available weather information to run the water balance
months = range(1, 12 + 1)
ini_yr_wp = years[0] + warmup_yrs

# Base flow of the previous month (mm)
bflow_ant = 10

In [11]:
# The recession constant (k) will be determined from hydrographs derived from the gathered streamflow datasets. For the rest of the world, k will be defined as 0.5
k_dict = {
    'MISS_07374525': 0.5,
    'ULUA_23': 0.5,
    'AMZN_17050001': 0.5,
    'NILE_1662100': 0.5,
    'YNGZ_2181900': 0.5,
    'MUDA_A4261110': 0.5,
    'DABE_6742900': 0.5,
    'DABE_42055': 0.5
}

In [12]:
# Initial soil water storage expressed as a fraction of water holding capacity [0-1]
ffcb_dict = {
    'MISS_07374525': 0.1,
    'ULUA_23': 0.1,
    'AMZN_17050001': 0.1,
    'NILE_1662100': 0.1,
    'YNGZ_2181900': 0.1,
    'MUDA_A4261110': 0.1,
    'DABE_6742900': 0.1,
    'DABE_42055': 0.1
}

In [13]:
st0 = Con(drain_areas_ids == 0, whc*ffcb_dict[id_dict[0]],
          Con(drain_areas_ids == 1, whc*ffcb_dict[id_dict[1]], 
              Con(drain_areas_ids == 2, whc*ffcb_dict[id_dict[2]], 
                  Con(drain_areas_ids == 3, whc*ffcb_dict[id_dict[3]],
                      Con(drain_areas_ids == 4, whc*ffcb_dict[id_dict[4]],
                          Con(drain_areas_ids == 5, whc*ffcb_dict[id_dict[5]], 
                              Con(drain_areas_ids == 6, whc*ffcb_dict[id_dict[6]], whc*ffcb_dict[id_dict[7]])))))))

RuntimeError: Unspecified error 

In [37]:
print('\n############################################################')
print('\t\tINITIAL VARIABLES')
print('\tPeriod to be executed: ' + str(years[0]) + '-' + str(years[-1]))
print('############################################################')

years = [1980]
months = [1]

for year in years:

    print("\n**Executing THORNTHWAITE AND MATHER model for " + str(year) + "**")

    ppt_nc_file = tc_ds + "\\TerraClimate_ppt_" + str(year) + ".nc"
    pet_nc_file  = tc_ds + "\\TerraClimate_pet_" + str(year) + ".nc"
    q_nc_file = tc_ds + "\\TerraClimate_q_" + str(year) + ".nc"

    nc_FP = arcpy.NetCDFFileProperties(ppt_nc_file)

    for month in months:

        print("\n\t*Executing water balance for month " + str(month) + "*\n")

        print("\tLoading precipitation, potential evapo, and runoff rasters from TerraClimate NetCDF files.....")

        # dimension_value = nc_FP.getDimensionValue("time", month-1)

        # ppt_name = "ppt_" + str(year) + "_" + str(month)
        # pet_name = "pet_" + str(year) + "_" + str(month)
        # q_name = "q_" + str(year) + "_" + str(month)

        # arcpy.MakeNetCDFRasterLayer_md(ppt_nc_file, "ppt", "lon", "lat", ppt_name, "", [["time", dimension_value]], "BY_VALUE")
        # arcpy.MakeNetCDFRasterLayer_md(pet_nc_file, "pet", "lon", "lat", pet_name, "", [["time", dimension_value]], "BY_VALUE")
        # arcpy.MakeNetCDFRasterLayer_md(q_nc_file, "q", "lon", "lat", q_name, "", [["time", dimension_value]], "BY_VALUE")

        # ppt_raster = arcpy.Raster(ppt_name)
        # pet_raster = arcpy.Raster(pet_name) * 0.1  # Scale factor: 0.1
        # q_raster = arcpy.Raster(q_name)

        print("\tPrecipitation, potential evapo., and runoff are loaded")

        print("\tCalculating effective precipitation......")
        eprec = ppt_raster - q_raster


############################################################
		INITIAL VARIABLES
	Period to be executed: 1980-1980
############################################################

**Executing THORNTHWAITE AND MATHER model for 1980**

	*Executing water balance for month 1*

	Loading precipitation, potential evapo, and runoff rasters from NetCDF files.....
	Precipitation, potential evapo., and runoff are loaded
	Calculating effective precipitation......
